# Function Calling with Llama 3.1: Getting started

In this example, we’ll demonstrate how to use function calling with the Llama 3.1 model (70B). We’ll integrate a simple function called calculateTool, which the model will be able to call when appropriate. The function attributes will be defined.

In [ ]:
#Install these required dependencies to run this notebook
!pip install langchain-community
!pip install python-dotenv==1.0.1

In [1]:
# Import libraries
import os
import sys
import json
from datetime import datetime
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool

from dotenv import load_dotenv

current_dir = os.getcwd()
repo_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(repo_dir)

from utils.model_wrappers.langchain_chat_models import ChatSambaNovaCloud

# Constants and Configuration
load_dotenv(os.path.join(repo_dir, '.env'), override=True)
SAMBANOVA_API_KEY = os.getenv("SAMBANOVA_API_KEY")
MODEL = "Meta-Llama-3.1-70B-Instruct"

In [2]:
# Tools definition

# get time tool definition
@tool
def get_time(kind: str = "both") -> str:
    """Returns current date, current time or both.
    Args:
        kind: date, time or both
    """
    if kind == "date":
        date = datetime.now().strftime("%m/%d/%Y")
        return f"Current date: {date}"
    elif kind == "time":
        time = datetime.now().strftime("%H:%M:%S")
        return f"Current time: {time}"
    else:
        date = datetime.now().strftime("%m/%d/%Y")
        time = datetime.now().strftime("%H:%M:%S")
        return f"Current date: {date}, Current time: {time}"
    
tools= [get_time]

# Define tool invocation method
def invoke_tools(tool_calls, messages):
    available_functions={tool.name:tool for tool in tools}
    for tool_call in tool_calls:
        selected_tool = available_functions[tool_call["name"]]
        tool_output = selected_tool.invoke(tool_call["args"])
        print(f"Tool output: {tool_output}")
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    return messages

In [ ]:
# Define tool calling model

llm = ChatSambaNovaCloud(
    sambanova_api_key = SAMBANOVA_API_KEY,
    model=MODEL,
    max_tokens=1024,
    temperature=0.7,
    top_k=1,
    top_p=0.01
)

llm_with_tools = llm.bind_tools(tools=tools)

In [4]:
# Call the function calling model

messages = [
    HumanMessage(
       content="I need to schedule a meeting for two weeks from today. Can you tell me the exact date of the meeting?"
    )
]

response = llm_with_tools.invoke(messages)
while len(response.tool_calls)>0:
    print(f"Intermediate model response: {response.tool_calls}")
    messages.append(response)
    messages = invoke_tools(response.tool_calls, messages)
    response = llm_with_tools.invoke(messages)

print(f"final response: {response.content}")

Intermediate model response: [{'name': 'get_time', 'args': {'kind': 'date'}, 'id': 'call_0958a08d37fc449590', 'type': 'tool_call'}]
Tool output: Current date: 11/13/2024
final response: Two weeks from today would be 11/27/2024.
